# Configure KBase Jupyter Dev Environment

In [1]:
%run probcommutil.py
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

python version 3.10.12
KBBaseModules 0.0.1
modelseedpy 0.3.3
cobrakbase 0.3.1
Output files printed to:/scratch/chenry/MicrobiomeNotebooks/grow/nboutput when using KBDevUtils.output_dir
ModelSEED: /scratch/shared//sdkmount/kb_sdk_home/run_local/workdir/tmp/


# Loading Clade Mapping Data

In [2]:
from pandas import read_excel
mag_taxa = read_excel("MAGs_toKBase_formodels.xlsx")
genera_to_track = ["Planktophila", "Methylopumilus", "Polynucleobacter", "Pirellula_B", "UBA3064", "UBA954"]
clade_mag_mapping = {"other":[]}
for index, row in mag_taxa.iterrows():
    if not isinstance(row["classification"], str):  continue
    found = False
    for genus in genera_to_track:
        if genus not in clade_mag_mapping:
            clade_mag_mapping[genus] = []
        if genus in row["classification"]:
            clade_mag_mapping[genus].append(row["user_genome"])
            found = True
            break
    if not found:
        clade_mag_mapping["other"].append(row["user_genome"])
util.kbdevutil.save("clade_mag_mapping", clade_mag_mapping)


# Pulling genomes and getting functions hash from all genomes

In [14]:
clade_mag_mapping = util.kbdevutil.load("clade_mag_mapping")
mag_list = {}
mag_refs = {}
primary_mags = util.msrecon.kbase_api.list_objects(106947, object_type="KBaseGenomes.Genome")
for mag in primary_mags:
    if mag[1][-5:] == ".RAST":
        name = mag[1][0:-5]
        mag_refs[name] = mag
        mag_list[name] = "primary"
other_mags = util.msrecon.kbase_api.list_objects(145226, object_type="KBaseGenomes.Genome")
for mag in other_mags:
    if mag[1][-5:] == ".RAST":
        name = mag[1][0:-5]
        if name not in mag_refs:
            mag_refs[name] = mag
            mag_list[name] = "secondary"
#Checking if mag is missing
allcount = 0
primcount = 0
for clade in clade_mag_mapping:
    for mag in clade_mag_mapping[clade]:
        allcount += 1
        newmag = mag.replace(".","_")+"_mag"
        if newmag in mag_refs:
            primcount += 1
            mag_refs[mag] = mag_refs[newmag]
        elif mag not in mag_refs:
            print("Missing mag: ", mag)
util.kbdevutil.save("mag_refs", mag_refs)
util.kbdevutil.save("mag_list", mag_list)
print("All",allcount)
print("Primary",primcount)

All 3825
Primary 2093


# Creating clade genomes and computing function probabilities

In [15]:
#Downloading genomes for each mag to make clade genome construction faster
clade_mag_mapping = util.kbdevutil.load("clade_mag_mapping")
mag_refs = util.kbdevutil.load("mag_refs")
for clade in clade_mag_mapping:
    for mag in clade_mag_mapping[clade]:
        if mag in mag_refs:
            ref = mag_refs[mag]
            genome = util.msrecon.kbase_api.get_object(ref[1],ref[6])
            util.kbdevutil.save(mag,genome)

In [16]:
#Now building clade genome objects and saving to file (will be loaded to KBase)
from cobrakbase.core.kbasegenome.genome import KBaseGenome
clade_mag_mapping = util.kbdevutil.load("clade_mag_mapping")
mag_refs = util.kbdevutil.load("mag_refs")
functions = {}
features = {}
feature_aliases = {}
feature_probabilities = {}
for clade in clade_mag_mapping:
    #Loading template genome
    with open("TemplateGenome.json", 'r') as f:
        template_genome = json.load(f)
    #Setting metadata
    template_genome["dna_size"] = 0
    template_genome["gc_content"] = 0.5
    template_genome["id"] = "GROW_"+clade
    template_genome["contig_ids"] = []
    template_genome["contig_lengths"] = []
    template_genome["num_contigs"] = 0
    template_genome["ontologies_present"] = {}
    template_genome["ontology_events"] = []
    template_genome["source"] = "GROW"
    template_genome["warnings"] = ["Artificial genome created to support clade modeling"]
    template_genome["features"] = []
    template_genome["cdss"] = []
    template_genome["source_id"] = "GROW"
    template_genome["molecule_type"] = "CladeGenome"
    template_genome["domain"] = "Bacteria"
    md5_list = []
    if clade not in functions:
        functions[clade] = {}
    for mag in clade_mag_mapping[clade]:
        if mag in mag_refs:
            ref = mag_refs[mag]
            genome = util.kbdevutil.load(mag)
            genome_functions = {}
            for ftr in genome["features"]:
                if "functions" in ftr:
                    for function in ftr["functions"]:
                        if function not in functions[clade]:
                            ftrid = clade+"_"+str(len(template_genome["contig_ids"])+1)
                            template_genome["contig_ids"].append(ftrid+".contig")
                            template_genome["contig_lengths"].append(len(ftr["dna_sequence"]))
                            template_genome["num_contigs"] += 1
                            template_genome["dna_size"] += len(ftr["dna_sequence"])
                            result = hashlib.md5(ftr["protein_translation"].encode())
                            md5 = result.hexdigest()
                            result = hashlib.md5(ftr["dna_sequence"].encode())
                            dnamd5 = result.hexdigest()
                            md5_list.append(md5)
                            functions[clade][function] = {"ftrid":ftrid, "probability":1}
                            features[ftrid] = {
                                "aliases": [],
                                "cdss": [
                                    ftrid+".CDS"
                                ],
                                "functions":[function],
                                "dna_sequence": ftr["dna_sequence"],
                                "dna_sequence_length": len(ftr["dna_sequence"]),
                                "id": ftrid,
                                "location": [
                                    [
                                        ftrid+".contig",
                                        1,
                                        "+",
                                        len(ftr["dna_sequence"])
                                    ]
                                ],
                                "md5": dnamd5,
                                "ontology_terms": {},
                                "protein_md5": md5,
                                "protein_translation": ftr["protein_translation"],
                                "protein_translation_length": len(ftr["protein_translation"]),
                                "warnings": []
                            }
                            cdsftr = features[ftrid].copy()
                            cdsftr["id"] = ftrid+".CDS"
                            cdsftr["parent_gene"] = ftrid
                            if "aliases" in ftr and len(ftr["aliases"]) >= 1:
                                features[ftrid]["aliases"].append(["gene",ftr["aliases"][0][1]])
                                if ftrid not in feature_aliases:
                                    feature_aliases[ftrid] = []
                                feature_aliases[ftrid].append([ftr["aliases"][0][1]])
                            template_genome["features"].append(features[ftrid])
                            template_genome["cdss"].append(cdsftr)
                        elif function not in genome_functions:#Don't want to count same function twice in a genome
                            ftrid = functions[clade][function]["ftrid"]
                            functions[clade][function]["probability"] += 1
                            if "aliases" in ftr and len(ftr["aliases"]) >= 1:
                                features[ftrid]["aliases"].append(["gene",ftr["aliases"][0][1]])
                                if ftrid not in feature_aliases:
                                    feature_aliases[ftrid] = []
                                feature_aliases[ftrid].append(ftr["aliases"][0][1])
                        genome_functions[function] = True
    for func in functions[clade]:
        functions[clade][func]["probability"] = functions[clade][func]["probability"]/len(clade_mag_mapping[clade])
        feature_probabilities[functions[clade][func]["ftrid"]] = functions[clade][func]["probability"]
    template_genome["feature_counts"] = {
        "CDS":len(template_genome["features"]),
        "gene":len(template_genome["features"]),
        "non_coding_features":0,
        "protein_encoding_gene":len(template_genome["features"])
    }                    
    md5_list.sort()
    result = hashlib.md5(";".join(md5_list).encode())
    template_genome["md5"] = result.hexdigest()
    #Writing FASTA
    ofile = open("Assemblies/"+clade+".fasta", "w")
    for func in functions[clade]:
        ofile.write(">" + functions[clade][func]["ftrid"] + "\n" +features[functions[clade][func]["ftrid"]]["dna_sequence"] + "\n")
    ofile.close()
    #Saving genome
    util.kbdevutil.save(clade+"_genome", template_genome)
util.kbdevutil.save("feature_aliases", feature_aliases)
util.kbdevutil.save("functions", functions)
util.kbdevutil.save("feature_probabilities", feature_probabilities)

# Loading assemblies to KBase

In [11]:
#Loading clade genome assemblies to KBase using SDK callbacks
afu = util.kbdevutil.afu_client()
#UBA954.fasta
clade_list = ["other","Planktophila","Methylopumilus","Polynucleobacter","Pirellula_B","UBA3064"]
for clade in clade_list:
    params = {
        'file': {
            'path': '/kb/module/work/tmp/'+clade+".fasta"
        },
        'workspace_id': 174284,
        'assembly_name': clade+".assembly"
    }
    result = afu.save_assembly_from_fasta2(params)

# Saving genomes to KBase

In [19]:
#Loading clade genomes to KBase using annotation API, which contains an SDK callback
from datetime import datetime
now = datetime.now()
timestamp = datetime.timestamp(now)
clade_list = ["UBA954","other","Planktophila","Methylopumilus","Polynucleobacter","Pirellula_B","UBA3064"]
anno = util.kbdevutil.anno_client()
anno.clients["GenomeFileUtil"] = util.kbdevutil.gfu_client()
for clade in clade_list:
    genome = util.kbdevutil.load(clade+"_genome")
    events = [{
        "id": "SSO",
        "event_id":"RAST:SSO:"+str(timestamp),
        "description": "RAST annotation of clade genome",
        "ontology_id": "SSO",
        "method": "RAST",
        "method_version": "1.9.5",
        "timestamp": str(timestamp),
        "ontology_terms": {},
    }]
    for ftr in genome["features"]:
        events[0]["ontology_terms"][ftr["id"]] = [{"term":"SSO:"+ftr["functions"][0]}]
    output = anno.add_annotation_ontology_events({
        "output_workspace":174284,
        "events":events,
        "overwrite_matching":True,
        "object":genome,
        "type":"KBaseGenomes.Genome",
        "output_name":clade+".genome",
        "save":1
    })

# Construct the clade models

In [ ]:
#"UBA954",
clade_list = ["other","Planktophila","Methylopumilus","Polynucleobacter","Pirellula_B","UBA3064"]
for clade in clade_list:
    util.msrecon.build_metabolic_models({
        "workspace":174284,
        "genome_refs":["174284/"+clade+".genome"],
        "run_gapfilling":True,
        "atp_safe":True,
        "forced_atp_list":[],
        "gapfilling_media_list":None,
        "suffix":".mdl",
        "core_template":"auto",
        "gs_template":"auto",
        "gs_template_ref":None,
        "core_template_ref":None,
        "template_reactions_only":True,
        "output_core_models":False,
        "automated_atp_evaluation":True,
        "atp_medias":[],
        "load_default_medias":True,
        "max_gapfilling":5,
        "gapfilling_delta":0,
        "return_model_objects":False,
        "return_data":False,
        "save_report_to_kbase":False,
        "change_to_complete":False,
        "gapfilling_mode":"Cumulative",
        "base_media":None,
        "compound_list":None,
        "base_media_target_element":"C"
    })

# Adding probabilities to clade models

In [ ]:
feature_probabilities = util.kbdevutil.load("feature_probabilities")
clade_list = ["UBA954","other","Planktophila","Methylopumilus","Polynucleobacter","Pirellula_B","UBA3064"]
for clade in clade_list:
    mdlutl = util.msrecon.get_model(clade+".genome.mdl",174284)
    for rxn in mdlutl.model.reactions:
        highest_prob = None
        for gene in rxn.genes:
            if gene.id in feature_probabilities:
                if highest_prob == None or feature_probabilities[gene.id] > highest_prob:
                    highest_prob = feature_probabilities[gene.id]
        if highest_prob != None:
            rxn.probability = highest_prob
            print(clade,rxn.id, highest_prob)
        else:
            print(clade,rxn.id, "No probability")
    util.msrecon.save_model(mdlutl,174284,clade+".genome.mdl")

# Printing genome alias translations

In [21]:
clade_list = ["UBA954","other","Planktophila","Methylopumilus","Polynucleobacter","Pirellula_B","UBA3064"]
grow_to_clade = {}
for clade in clade_list:
    genome = util.kbdevutil.load(clade+"_genome")
    for ftr in genome["features"]:
        if "aliases" in ftr:
            for alias in ftr["aliases"]:
                grow_to_clade[alias[1]] = [ftr["id"],clade]
util.kbdevutil.save("grow_to_clade", grow_to_clade)

# Translating abundances
# DONE - but need to explore other coefficients of "other"

In [ ]:
clade_mag_mapping = util.kbdevutil.load("clade_mag_mapping")
mag_list = util.kbdevutil.load("mag_list")
mag_abundances = {}
condition_list = []
abundance = read_csv("MetaT/norm.counts.rpk_edger.bins_mean.csv")
for column_name, series in abundance.iteritems():
    if column_name != "rn" and column_name != "Unnamed: 0":
        condition_list.append(column_name)
for index, row in abundance.iterrows():
    if row["rn"] in mag_list:
        if row["rn"] not in mag_abundances:
            mag_abundances[row["rn"]] = {}
        for condition in condition_list:
            if condition in row:
                mag_abundances[row["rn"]][condition] = row[condition]
    else:
        print("MAG in abundance mismatches other data: ", row["rn"])
#Adding up all abundances for mags in clades
clade_abundances = {}
for clade in clade_mag_mapping:
    clade_abundances[clade] = {}
    for mag in clade_mag_mapping[clade]:
        if mag in mag_abundances:
            for condition in mag_abundances[mag]:
                if condition not in clade_abundances[clade]:
                    clade_abundances[clade][condition] = 0
                clade_abundances[clade][condition] += mag_abundances[mag][condition]
        else:
            print("MAG in clade ",clade," not found in abundance data:", mag)
    if clade == "other":
        for condition in condition_list:
            clade_abundances[clade][condition] = 0.001*clade_abundances[clade][condition]
#Normalizing
for condition in condition_list:
    total = 0
    for clade in clade_abundances:
        total += clade_abundances[clade][condition]
    for clade in clade_abundances:
        clade_abundances[clade][condition] = clade_abundances[clade][condition]/total
util.kbdevutil.save("condition_list", condition_list)
util.kbdevutil.save("mag_abundances", mag_abundances)
util.kbdevutil.save("clade_abundances", clade_abundances)

# Translating metagenome expression to clade expression

In [21]:
condition_list = util.kbdevutil.load("condition_list")
grow_to_clade = util.kbdevutil.load("grow_to_clade")
util.extract_metaT_data("MetaT-20240116T195841Z-001.zip")
metaT = read_csv("MetaT/norm.counts.rpk_edger_geTMM.csv")
clade_list = ["UBA954","other","Planktophila","Methylopumilus","Polynucleobacter","Pirellula_B","UBA3064"]
clade_records = {}
feature_entries = {}
for clade in clade_list:
    clade_records[clade] = []
    feature_entries[clade] = {}
for index, row in metaT.iterrows():
    geneid = row["Unnamed: 0"]
    if geneid in grow_to_clade:
        clade = grow_to_clade[geneid][1]
        ftrid = grow_to_clade[geneid][0]
        if ftrid not in feature_entries[clade]:
            feature_entries[clade][ftrid] = {"id":ftrid}
            clade_records[clade].append(feature_entries[clade][ftrid])
        for column_name in condition_list:
            if column_name not in feature_entries[clade][ftrid]:
                feature_entries[clade][ftrid][column_name] = 0
            feature_entries[clade][ftrid][column_name] += row[column_name]
for clade in clade_list:
    df = DataFrame.from_records(clade_records[clade])
    df.to_csv("MetaT/"+clade+"_expression.csv")
util.kbdevutil.save("feature_entries", feature_entries)

# Building and saving community model

In [ ]:
import cobra
clade_list = ["Planktophila","Methylopumilus","Polynucleobacter","Pirellula_B","UBA3064","UBA954","other"]
member_models = []
template_refs = []
genome_refs = []
for clade in clade_list:
    model = util.msrecon.get_model(clade+".genome.mdl",174284)
    template_refs.append(model.model.template_ref)
    genome_refs.append(model.model.genome_ref)
    member_models.append(model.model)
comm_model = MSCommunity.build_from_species_models(
    member_models,
    mdlid="GROWCladeCommunityModel", 
    name="GROWCladeCommunityModel",
    names=clade_list,
    abundances=None,
    #basemodel=base_model
)
#Saving probabilities
community_rxn_probs = {}
for rxn in comm_model.model.reactions:
    if hasattr(rxn, "probability"):
        community_rxn_probs[rxn.id] = rxn.probability
util.kbdevutil.save("community_rxn_probs", community_rxn_probs)
cobra.io.save_json_model(comm_model.model, 'GROW_community_mdl.json')

# Creating and saving model with abundances of each sample\
# DONOT RUN - ABUNDANCES NOT CACHED

In [ ]:
clade_list = ["Planktophila","Methylopumilus","Polynucleobacter","Pirellula_B","UBA3064","UBA954","other"]
clade_abundances = util.kbdevutil.load("clade_abundances")
model = util.msrecon.get_model("BaseCommunity.mdl",174284)
comm_model = MSCommunity(
    model=model,
    ids=clade_list,
    abundances=None,
    kinetic_coeff=800,
    lp_filename=None,
    printing=False
)
condition_to_community_model = {}
for condition in condition_list:
    abundance_hash = {}
    for clade in clade_abundances:
        if condition in clade_abundances[clade]:
            abundance_hash[clade] = clade_abundances[clade][condition]
    comm_model.set_abundance()
    util.msrecon.save_model(comm_model,174284,condition[0:-32].".commdl")
    condition_to_community_model[condition] = condition[0:-32]+".commdl"
util.kbdevutil.save("condition_to_community_model", condition_to_community_model)

# Simulating community model in each condition

In [2]:
from optlang.symbolics import Zero, add
import cobra
loadedmodel = cobra.io.load_json_model('GROW_community_mdl.json')
loadedmodel.solver = 'gurobi'
dipeptide_exchanges = [
    "EX_cpd11591_e0",
    "EX_cpd11589_e0",
    "EX_cpd15605_e0",
    "EX_cpd11588_e0",
    "EX_cpd11583_e0",
    "EX_cpd11580_e0",
    "EX_cpd11593_e0",
    "EX_cpd11585_e0",
    "EX_cpd11586_e0",
    "EX_cpd15604_e0",
    "EX_cpd11581_e0",
    "EX_cpd01017_e0",
    "EX_cpd11590_e0",
    "EX_cpd11592_e0",
    "EX_cpd11584_e0",
    "EX_cpd00731_e0",
    "EX_cpd15603_e0",
    "EX_cpd11587_e0",
    "EX_cpd11582_e0",
    "EX_cpd15606_e0",
    "EX_cpd03424_e0",
    "EX_cpd00423_e0",
    "EX_cpd00080_e0",
    "EX_cpd02233_e0",
    "EX_cpd00355_e0",
    "EX_cpd00235_e0",
    "EX_cpd00079_e0",
    "EX_cpd01912_e0"
]
clade_list = ["Planktophila","Methylopumilus","Polynucleobacter","Pirellula_B","UBA3064","UBA954","other"]
community_rxn_probs = util.load("community_rxn_probs")
clade_abundances = util.load("clade_abundances")
condition_list = util.load("condition_list")
feature_entries = util.load("feature_entries")
min_prob = 0.05
min_expression = 0
exp_coef = -100
prob_exp = 1
ex_weight = 1
oxygen_uptake = 10
kinetics_coefs = [400,700,1000]
clade_expression_hash = {}
for kinetics_coef in kinetics_coefs:
    output = {
        "kinetics_coef":kinetics_coef,
        "conditions":{}
    }
    model_copy = cobra.io.json.from_json(cobra.io.json.to_json(loadedmodel))
    current_comm_model = MSCommunity(
        model=model_copy,
        names=clade_list
    )
    mdlutl = current_comm_model.mdlutl
    pkgmgr = MSPackageManager.get_pkg_mgr(mdlutl)
    media = util.msrecon.get_media("KBaseMedia/Complete")
    for condition in condition_list:
        clade_expression_hash[condition] = {}
        output["conditions"][condition] = {"solution":{}}
        pkgmgr.getpkg("ObjConstPkg").clear()
        #Setting abundances from condition abundance data
        abundance_hash = {}
        for clade in clade_abundances:
            if condition in clade_abundances[clade]:
                abundance_hash[clade] = clade_abundances[clade][condition]
        current_comm_model.set_abundance(abundance_hash)
        #Setting media
        pkgmgr.getpkg("KBaseMediaPkg").build_package(media)
        #Adding commkinetic constraints
        pkgmgr.getpkg("CommKineticPkg").build_package(kinetics_coef, current_comm_model)
        #Adding elemental uptake constraints
        pkgmgr.getpkg("ElementUptakePkg").build_package({"C": 300})
        for rxn in loadedmodel.reactions:
            if "EX_" == rxn.id[0:3]:
                currrxn = current_comm_model.model.reactions.get_by_id(rxn.id)
                if rxn.id in dipeptide_exchanges:
                    currrxn.lower_bound = 0
                    currrxn.upper_bound = 0
                elif rxn.id == "EX_cpd00007_e0":
                    currrxn.lower_bound = -1*oxygen_uptake
                else:
                    if currrxn.lower_bound < 0:
                        currrxn.lower_bound = -1000
                    if currrxn.upper_bound > 0:
                        currrxn.upper_bound = 1000
        #Maximize biomass production
        #mdlutl.model.objective = "bio1"
        mdlutl.model.objective = mdlutl.model.problem.Objective(Zero, direction="max")
        mdlutl.model.objective.set_linear_coefficients({mdlutl.model.reactions.bio1.forward_variable: 1})
        current_comm_model.print_lp("Run.lp")
        output["conditions"][condition]["max_growth"] = mdlutl.model.slim_optimize()
        output["conditions"][condition]["carbon_uptake"] = pkgmgr.getpkg("ElementUptakePkg").variables["elements"]["C"].primal
        print("Initial growth:", output["conditions"][condition]["max_growth"])
        print("Carbon uptake:",output["conditions"][condition]["carbon_uptake"])
        if str(output["conditions"][condition]["max_growth"]) == "nan":
            print("Skipping condition due to infeasibility", condition)
            continue
        #Constraining to 90% of community biomass
        pkgmgr.getpkg("ObjConstPkg").clear()
        pkgmgr.getpkg("ObjConstPkg").build_package(output["conditions"][condition]["max_growth"] * 0.9, None)
        #Creating minimal probability objective
        coef = {}
        for rxn in loadedmodel.reactions:
            if "rxn" == rxn.id[0:3]:
                probability = community_rxn_probs[rxn.id]
                currrxn = current_comm_model.model.reactions.get_by_id(rxn.id)
                coef.update(
                    {
                        currrxn.forward_variable: max(
                            min_prob, (1 - float(probability) ** prob_exp)
                        )
                    }
                )
                coef.update(
                    {
                        currrxn.reverse_variable: max(
                            min_prob, (1 - float(probability) ** prob_exp)
                        )
                    }
                )
            elif "EX_" == rxn.id[0:3]:
                currrxn = current_comm_model.model.reactions.get_by_id(rxn.id)
                coef.update({currrxn.forward_variable: ex_weight})
                coef.update({currrxn.reverse_variable: ex_weight})
        #Adding expression data to minimum probability objective
        for clade in feature_entries:
            total = 0
            for ftr in feature_entries[clade]:
                total += feature_entries[clade][ftr][condition]
            for ftr in feature_entries[clade]:
                feature_entries[clade][ftr][condition] = feature_entries[clade][ftr][condition]/total
        filter = {}
        for rxn in mdlutl.model.reactions:
            highest_exp = 0
            for gene in rxn.genes:
                array = gene.id.split("_")
                array.pop()
                clade = "_".join(array)
                if gene.id in feature_entries[clade] and condition in feature_entries[clade][gene.id]:
                    if feature_entries[clade][gene.id][condition] > highest_exp:
                        highest_exp = feature_entries[clade][gene.id][condition]
                    if feature_entries[clade][gene.id][condition] > 0:
                        if clade not in clade_expression_hash[condition]:
                            clade_expression_hash[condition][clade] = 0
                        clade_expression_hash[condition][clade] += 1
            if highest_exp > min_expression and exp_coef != 0:
                if rxn.upper_bound > 0:
                    coef.update(
                        {
                            rxn.forward_variable: exp_coef * highest_exp
                        }
                    )
                if rxn.lower_bound < 0:
                    coef.update(
                        {
                            rxn.reverse_variable: exp_coef * highest_exp
                        }
                    )
                if rxn.upper_bound > 0 and rxn.lower_bound < 0:
                    filter[rxn.id] = 1
        #Setting the objective
        if condition == condition_list[0] and exp_coef != 0:
            pkgmgr.getpkg("RevBinPkg").build_package(filter=filter)
        mdlutl.model.objective = mdlutl.model.problem.Objective(Zero, direction="min")
        mdlutl.model.objective.set_linear_coefficients(coef)
        with open("lp/"+condition+".lp", "w") as out:
            out.write(str(mdlutl.model.solver))
        #Solving the LP
        solution = mdlutl.model.optimize()
        output["conditions"][condition]["objective"] = solution.objective_value
        output["conditions"][condition]["solution"][rxn.id] = solution.fluxes[rxn.id]
        for rxn in mdlutl.model.reactions:
            output["conditions"][condition]["solution"][rxn.id] = solution.fluxes[rxn.id]
    util.save(str(oxygen_uptake)+"o2_"+str(kinetics_coef)+"_"+str(abs(exp_coef))+"exp_output", output)
    filename = util.output_dir+"/"+str(oxygen_uptake)+"o2_"+str(kinetics_coef)+"_"+str(abs(exp_coef))+"exp.csv"
    with open(filename, 'w') as f:
        records = [
            {"id":"max_growth"},
            {"id":"carbon_uptake"},
            {"id":"objective"},
        ]
        rxn_record_hash = {}
        for condition in output["conditions"]:
            records[0][condition] = output["conditions"][condition]["max_growth"]
            records[1][condition] = output["conditions"][condition]["carbon_uptake"]
            records[2][condition] = output["conditions"][condition]["objective"]
            solution = output["conditions"][condition]["solution"]
            for rxn in solution:
                if rxn not in rxn_record_hash:
                    rxn_record_hash[rxn] = {}
                    rxn_record_hash[rxn]["id"] = rxn
                    records.append(rxn_record_hash[rxn])
                rxn_record_hash[rxn][condition] = solution[rxn]
    df = DataFrame.from_records(records)
    df.to_csv(filename)

Set parameter TSPort to value 27070


1714778302.6406472 INFO: Set parameter TSPort to value 27070


Set parameter TokenServer to value "lic-vmw-01.cels.anl.gov"


1714778302.642588 INFO: Set parameter TokenServer to value "lic-vmw-01.cels.anl.gov"
1714778341.6228905 INFO: Making atp hydrolysis reaction for species: Planktophila


bio1 {<Metabolite cpd11416_c0 at 0x7f8d4ba27370>: 1, <Metabolite cpd11416_c1 at 0x7f8d4bc36ad0>: -0.14285714285714285, <Metabolite cpd11416_c2 at 0x7f8d4bc77ca0>: -0.14285714285714285, <Metabolite cpd11416_c3 at 0x7f8d4bccb4f0>: -0.14285714285714285, <Metabolite cpd11416_c4 at 0x7f8d4bb20af0>: -0.14285714285714285, <Metabolite cpd11416_c5 at 0x7f8d4bb62b90>: -0.14285714285714285, <Metabolite cpd11416_c6 at 0x7f8d4bbb9450>: -0.14285714285714285, <Metabolite cpd11416_c7 at 0x7f8d4ba27340>: -0.14285714285714285}
SK_cpd11416_c0 {<Metabolite cpd11416_c0 at 0x7f8d4ba27370>: -1}


...

4 Pirellula_B


1714783310.053991 INFO: Making atp hydrolysis reaction for species: UBA954


5 UBA3064


1714783310.4969 INFO: Making atp hydrolysis reaction for species: other


6 UBA954
7 other
Initial growth: 5.899905005769685
Carbon uptake: 300.0
Initial growth: 7.420326882767026
Carbon uptake: 300.0
Initial growth: 8.44526940863228
Carbon uptake: 300.0
Initial growth: 6.012737342370258
Carbon uptake: 300.0
Initial growth: 3.927583124742292
Carbon uptake: 300.0
Initial growth: 5.4682822777469875
Carbon uptake: 300.0
Initial growth: 5.2141805793808995
Carbon uptake: 300.0
Initial growth: 4.229851030773385
Carbon uptake: 300.0
Initial growth: 5.72221951871206
Carbon uptake: 300.0
Initial growth: 4.015065947102115
Carbon uptake: 300.0
Initial growth: 5.842542937263233
Carbon uptake: 300.0
Initial growth: 5.5375594033356315
Carbon uptake: 300.0
Initial growth: 4.262257555815634
Carbon uptake: 300.0
Initial growth: 4.700898855877059
Carbon uptake: 300.0
Initial growth: 7.278012029275242
Carbon uptake: 300.0
Initial growth: 4.266380889032017
Carbon uptake: 300.0
Initial growth: 4.5744235745351185
Carbon uptake: 300.0
Initial growth: 4.879483212416167
Carbon uptak